<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/RAG_Using_Unstructured%2C_Llama3_via_Groq%2C_Qdrant_%26_LangChain_09_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangChain
Unstructured
LangSmith
Qdrant Cloud
Groq API
Llama3 via Groq API
Fastembed

In [2]:
%%capture
%pip install "unstructured[all-docs]" unstructured-client watermark langchain-groq langchain fastembed qdrant_client python-dotenv

In [3]:
!pip install onnxruntime==1.19.2 #must be less than 1.20.0

In [4]:
%load_ext watermark

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements

In [7]:
%watermark --iversions

unstructured: 0.16.5



In [8]:
import unstructured.partition

help(unstructured.partition)

Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common (package)
    csv
    doc
    docx
    email
    epub
    html (package)
    image
    json
    md
    model_init
    msg
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    /usr/local/lib/python3.10/dist-packages/unstructured/partition/__init__.py




In [9]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [10]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [11]:
!pip install -U nltk

In [12]:
import nltk
nltk.download('punkt', download_dir='/root/nltk_data')
nltk.download('punkt_tab', download_dir='/root/nltk_data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
import nltk
print(nltk.__version__) #must be 3.9.1

3.9.1


In [14]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = "/content/gpt4all.pdf"
#path = "images"

# Extract images, tables, and chunk text
pdf_elements = partition_pdf(
    filename=filename,
    extract_images_in_pdf=False,
    strategy = "hi_res",
    hi_res_model_name="yolox",
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=3000,
    #new_after_n_chars=3800,
    combine_text_under_n_chars=200,
    #extract_image_block_output_dir=path,
)

In [15]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 15,
 "<class 'unstructured.documents.elements.TableChunk'>": 2}

In [16]:
element_dict = [el.to_dict() for el in pdf_elements]

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'CompositeElement', 'Table'}


In [17]:
# Extract images, tables, and chunk text
pdf_elements = partition_pdf(
    filename=filename,
    extract_images_in_pdf=False,
    strategy = "hi_res",
    hi_res_model_name="yolox",
    infer_table_structure=True,
    #chunking_strategy="by_title",
    max_characters=3000,
    #new_after_n_chars=3800,
    combine_text_under_n_chars=200,
    #extract_image_block_output_dir=path,
)

for element in pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
print(category_counts)

element_dict = [el.to_dict() for el in pdf_elements]

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{"<class 'unstructured.documents.elements.CompositeElement'>": 15, "<class 'unstructured.documents.elements.TableChunk'>": 2, "<class 'unstructured.documents.elements.Text'>": 14, "<class 'unstructured.documents.elements.Header'>": 1, "<class 'unstructured.documents.elements.Title'>": 34, "<class 'unstructured.documents.elements.EmailAddress'>": 4, "<class 'unstructured.documents.elements.NarrativeText'>": 34, "<class 'unstructured.documents.elements.Footer'>": 1, "<class 'unstructured.documents.elements.Image'>": 5, "<class 'unstructured.documents.elements.FigureCaption'>": 2, "<class 'unstructured.documents.elements.Table'>": 1, "<class 'unstructured.documents.elements.ListItem'>": 29}
{'UncategorizedText', 'FigureCaption', 'Header', 'ListItem', 'Table', 'Title', 'Footer', 'NarrativeText', 'Image', 'EmailAddress'}


In [18]:
len(pdf_elements)

125

In [19]:
pdf_elements[0].to_dict()

{'type': 'UncategorizedText',
 'element_id': '27112c1430a5b5622e963badf8ded44e',
 'text': '3',
 'metadata': {'coordinates': {'points': ((45.388888888888886,
     732.8055555555557),
    (45.388888888888886, 760.5833333333335),
    (100.94444444444446, 760.5833333333335),
    (100.94444444444446, 732.8055555555557)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-11-09T14:56:20',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '/content',
  'filename': 'gpt4all.pdf'}}

In [20]:
tables = [el for el in pdf_elements if el.category == "Table"]

In [21]:
table_html = tables[0].metadata.text_as_html

In [22]:
table_html = tables[0].metadata.text_as_html
from io import StringIO
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

<table>
  <thead>
    <tr>
      <th>Model</th>
      <th>BoolQ</th>
      <th>PIQA</th>
      <th>HellaSwag</th>
      <th>WinoG.</th>
      <th>ARC-e</th>
      <th>ARC-c</th>
      <th>OBQA</th>
      <th>Avg.</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>GPT4AII-J 6B v1.0*</td>
      <td>73.4</td>
      <td>74.8</td>
      <td>63.4</td>
      <td>64.7</td>
      <td>54.9</td>
      <td>36</td>
      <td>40.2</td>
      <td>58.2</td>
    </tr>
    <tr>
      <td>GPT4AIl-J v1.1-breezy*</td>
      <td>74</td>
      <td>75.1</td>
      <td>63.2</td>
      <td>63.6</td>
      <td>55.4</td>
      <td>34.9</td>
      <td>38.4</td>
      <td>57.8</td>
    </tr>
    <tr>
      <td>GPT4AIl-J v1.2-jazzy*</td>
      <td>74.8</td>
      <td>74.9</td>
      <td>63.6</td>
      <td>63.8</td>
      <td>56.6</td>
      <td>35.3</td>
      <td>41</td>
      <td>58.6</td>
    </tr>
    <tr>
      <td>GPT4AII-J v1.3-groovy*</td>
      <td>73.6</td>
      <td>74.3</td>
      <td>63.8</td>
    

In [23]:
# Find the element with text "References" and category "Title"
reference_title = [
    el for el in pdf_elements
    if el.text == "References"
    and el.category == "Title"
][0]

In [24]:
reference_title.to_dict()

{'type': 'Title',
 'element_id': 'd3f115969fa159c8ae83287b2de7a62e',
 'text': 'References',
 'metadata': {'detection_class_prob': 0.8571382164955139,
  'coordinates': {'points': ((196.85, 199.78396606445312),
    (196.85, 235.40411376953125),
    (351.13720722222223, 235.40411376953125),
    (351.13720722222223, 199.78396606445312)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-11-09T14:56:20',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 5,
  'file_directory': '/content',
  'filename': 'gpt4all.pdf'}}

In [25]:
# Get the ID of the reference title element
references_id = reference_title.id

In [26]:
references_id

'd3f115969fa159c8ae83287b2de7a62e'

In [27]:
for element in pdf_elements:
    if element.metadata.parent_id == references_id:
        print(element)
        break

Nomic AI. 2023. Atlas. https://atlas.nomic.ai/.


In [28]:
# Filter out elements with a parent_id matching references_id
pdf_elements = [el for el in pdf_elements if el.metadata.parent_id != references_id]

In [30]:
headers = [el for el in pdf_elements if el.category == "Header"]

In [31]:
len(headers)

1

In [33]:
headers[0].to_dict()

{'type': 'Header',
 'element_id': '5fea6da62d2667f5b006561b6ada0c46',
 'text': '2 v o N 6 ] L C . s c [ 1 v 1 3 9 4 0 . 1 1 3 2',
 'metadata': {'detection_class_prob': 0.5669687390327454,
  'coordinates': {'points': ((45.388888888888886, 808.9898071289062),
    (45.388888888888886, 1545.0775146484375),
    (100.94444444444446, 1545.0775146484375),
    (100.94444444444446, 808.9898071289062)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-11-09T14:56:20',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '/content',
  'filename': 'gpt4all.pdf'}}

In [34]:
# Filters out elements from the `pdf_elements` list that have the category "Header".
pdf_elements = [el for el in pdf_elements if el.category != "Header"]

In [35]:
len(pdf_elements)

92

In [36]:
# lets again see some random index
pdf_elements[33].to_dict()

{'type': 'NarrativeText',
 'element_id': '26b658df23e74cc6c837451f59180d06',
 'text': 'Large language models (LLMs) have recently achieved human-level performance on a range of professional and academic benchmarks. The accessibility of these models has lagged behind their performance. State-of-the-art LLMs re- quire costly infrastructure; are only accessible via rate-limited, geo-locked, and censored web interfaces; and lack publicly available code and technical reports.',
 'metadata': {'detection_class_prob': 0.9421735405921936,
  'coordinates': {'points': ((243.40277777777777, 920.7620044444442),
    (243.40277777777777, 1214.1053377777775),
    (761.8583984375, 1214.1053377777775),
    (761.8583984375, 920.7620044444442)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-11-09T14:56:20',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'parent_id': 'ef299cc63d1bd25691568b2eb9efea97',
  'file_directo

## Preprocess the markdown file

In [37]:
filename_md = "/content/uber_10q_march_2022.md"

In [38]:
md_elements = partition_md(filename=filename_md)

In [39]:
# lets again see some random index
md_elements[33].to_dict(), pdf_elements[33].to_dict()

({'type': 'Title',
  'element_id': 'd7e7b8d880ae1ff8fd2f389fd1f21329',
  'text': 'PART I - FINANCIAL INFORMATION',
  'metadata': {'category_depth': 1,
   'languages': ['eng'],
   'file_directory': '/content',
   'filename': 'uber_10q_march_2022.md',
   'filetype': 'text/markdown',
   'last_modified': '2024-11-09T15:22:05',
   'parent_id': '868036a9751d25abada9c8d4ee8a0694'}},
 {'type': 'NarrativeText',
  'element_id': '26b658df23e74cc6c837451f59180d06',
  'text': 'Large language models (LLMs) have recently achieved human-level performance on a range of professional and academic benchmarks. The accessibility of these models has lagged behind their performance. State-of-the-art LLMs re- quire costly infrastructure; are only accessible via rate-limited, geo-locked, and censored web interfaces; and lack publicly available code and technical reports.',
  'metadata': {'detection_class_prob': 0.9421735405921936,
   'coordinates': {'points': ((243.40277777777777, 920.7620044444442),
     (243.

In [40]:
len(pdf_elements), len(md_elements)

(92, 1510)

In [41]:
elements = chunk_by_title(pdf_elements + md_elements) # you can play around with the chunk_by_title arguments

In [42]:
len(elements)

737

In [43]:
pdf_elements[0].to_dict()

{'type': 'UncategorizedText',
 'element_id': '27112c1430a5b5622e963badf8ded44e',
 'text': '3',
 'metadata': {'coordinates': {'points': ((45.388888888888886,
     732.8055555555557),
    (45.388888888888886, 760.5833333333335),
    (100.94444444444446, 760.5833333333335),
    (100.94444444444446, 732.8055555555557)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-11-09T14:56:20',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '/content',
  'filename': 'gpt4all.pdf'}}

In [44]:
pdf_elements[1].to_dict()

{'type': 'UncategorizedText',
 'element_id': 'f1d391632cea8ffe4bc7ec34590c6f6b',
 'text': '2023',
 'metadata': {'coordinates': {'points': ((51.0, 735.0),
    (51.0, 840.0),
    (88.0, 840.0),
    (88.0, 735.0)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-11-09T14:56:20',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '/content',
  'filename': 'gpt4all.pdf'}}

In [45]:
elements[0].to_dict()

{'type': 'CompositeElement',
 'element_id': 'ecbe1f24-e1d5-4101-9c4f-8e16b03ae648',
 'text': '3\n\n2023\n\n2\n\n0\n\n:\n\nv\n\narXiv\n\ni\n\nX\n\nr\n\na\n\nGPT4All: An Ecosystem of Open Source Compressed Language Models\n\nYuvanesh Anand\n\nNomic AI\n\nyuvanesh@nomic.ai\n\nZach Nussbaum\n\nNomic AI\n\nzach@nomic.ai\n\nAdam Treat\n\nNomic AI\n\nadam@nomic.ai\n\nAaron Miller Nomic AI aaron@nomic.ai\n\nRichard Guo\n\nNomic AI\n\nrichard@nomic.ai\n\nBen Schmidt Nomic AI ben@nomic.ai\n\nGPT4All Community\n\nPlanet Earth\n\nBrandon Duderstadt∗\n\nNomic AI brandon@nomic.ai\n\nAndriy Mulyar∗\n\nNomic AI andriy@nomic.ai',
 'metadata': {'file_directory': '/content',
  'filename': 'gpt4all.pdf',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'last_modified': '2024-11-09T14:56:20',
  'page_number': 1,
  'orig_elements': 'eJzlm1tvG7kVx7/KQM/ZWd4OL3mq2wbbAk0a7KbAdrOBcUge2oPqYoxGabzBfveSM5pYtrS1osUAazlv/zOkJc2P50Yy7z/PaE4LWnaXTZy9rGbCcC4CV5IheNBCkNPSY0w2UlSKZi+q2YI6jNhhHv95FlarNjZL7Gjd6zn

In [46]:
!pip install langchain_community

In [47]:
import os
from langchain_core.documents import Document
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Qdrant

In [48]:
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [49]:
len(documents)

737

In [50]:
documents[0]

Document(metadata={'file_directory': '/content', 'filename': 'gpt4all.pdf', 'filetype': 'application/pdf', 'last_modified': '2024-11-09T14:56:20', 'page_number': 1, 'orig_elements': 'eJzlm1tvG7kVx7/KQM/ZWd4OL3mq2wbbAk0a7KbAdrOBcUge2oPqYoxGabzBfveSM5pYtrS1osUAazlv/zOkJc2P50Yy7z/PaE4LWnaXTZy9rGbCcC4CV5IheNBCkNPSY0w2UlSKZi+q2YI6jNhhHv95FlarNjZL7Gjd6znerjbd5TU1V9ddtggpXZ6zNf+3id11tnINKltvVs2yK/Pev1dQZ4uRorYfXlSj1KzWRXLGandA98OzYba+XXe0KL/gbfOJ5j/cYKDZr/lBauZ0GZuWQrdqb8uAb8Nq2eVfPNs+XeKCiv3qplM4n9c3MY2Putub/hHe3MybgF2zWn67fTzH5dUGr/pf/X5Gy6vZh9667i4Xq9ikhob3yYT6hvNvmHvH1UvQLwUrs2/yzMvlZuGpLW+jfNGOPpX3NZPl+fjB/1rmT6WrVdv8QvFdGZGHPmSWeJSOaykCoU2JlA+GglTgWNBJ+8mYAa9ZgQA1K1AGaRUbpLUH5DD4vIjlGSdAM/lzHEXtBdMpGeWUQqEiWieU8HJqRxs9aSszHbXrWA91P/zMsH01MxUjD9Fq9IknEa3R4Hk0yNEDamRTMxuhDNJyXfMdRg/1MPy8mLGvZxZMUiBtsDExBhaS04kIPRdKAbqJmfH80bW8g8YBZG12KB0w9DOePrfe0t6RgESoI2oRhdYmgWdMg+Ze53fPgtKzXdAvvxq0QRnI8ly3BJ+CiTEwq4JMQUeNgcfJQY8cR215Dfe47hn6Gc8c9Mdd0O+abk6H4HKunMsp03Kfshtr4pGAPCqBqI2ZLluCKFVLRqW2Nc6gt

In [51]:
# from dotenv import load_dotenv
# load_dotenv()

In [79]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
QDRANT_KEY = userdata.get('QDRANT_KEY')
QDRANT_URL = userdata.get('QDRANT_URL')
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')

In [53]:
embeddings = FastEmbedEmbeddings()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [54]:
# this will take some time, patience is the key :)
vectorstore = Qdrant.from_documents(documents=documents,
                                    embedding = embeddings,
                                    url = QDRANT_URL,
                                    collection_name="rag",
                                    prefer_grpc=True,
                                    api_key=QDRANT_KEY)

In [55]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [67]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import create_history_aware_retriever
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [58]:
# template = """You are an AI assistant for answering questions about the GPT4All paper and Quarterly Report Pursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934 for the quarterly period ended March 31, 2022.
# You are given the following extracted parts of a long document and a question. Provide a conversational answer.
# If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
# Question: {question}
# =========
# {context}
# =========
# Answer in Markdown:"""
# prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [68]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

In [69]:
llm = ChatGroq(temperature=0,model_name="llama3-8b-8192", api_key=GROQ_API_KEY)

In [70]:
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [81]:
rag_chain.invoke({
    "input": "What was the net loss including non-controlling interests of Uber in 2021", #line 533
    "chat_history": []
})["answer"]

'According to the provided context, the net loss including non-controlling interests of Uber in 2021 was $(122) million.'

In [73]:
# hybrid search in action
filter_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1, "filter": {"source": "gpt4all.pdf"}}
)

In [74]:
filter_rag_chain = create_retrieval_chain(filter_retriever, question_answer_chain)

In [82]:
filter_rag_chain.invoke({
    "input": "How was GPT4All-Snoozy developed ?",
    "chat_history": [],
     "filter": filter,
})["answer"]

"GPT4All-Snoozy was developed using a similar procedure as previous GPT4All models, but with a few key modifications. It used the LLaMA-13B base model and incorporated Dolly's training data into its training mix. The model was trained on a set of 739,259 prompt-response pairs after data curation and deduplication."

In [80]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY= LANGCHAIN_API_KEY
LANGCHAIN_PROJECT="rag-llama3"